In [80]:
import numpy as np
import pyvisa as visa
import time
from matplotlib import pyplot as plt
import pandas as pd

In [2]:
rm = visa.ResourceManager()

In [3]:
rm.list_resources()

('USB0::0x0699::0x0368::C017041::INSTR',
 'USB0::0x0699::0x0353::2139465::INSTR',
 'ASRL1::INSTR',
 'ASRL10::INSTR')

In [4]:
resource_name_f=  'USB0::0x0699::0x0353::2139465::INSTR'
resource_name_o = 'USB0::0x0699::0x0368::C017041::INSTR'

osci = rm.open_resource(resource_name_o)
fungen = rm.open_resource(resource_name_f)

In [5]:
osci.query('*IDN?')

'TEKTRONIX,TBS 1052B-EDU,C017041,CF:91.1CT FV:v4.06\n'

In [6]:
fungen.query('*IDN?')

'TEKTRONIX,AFG1022,2139465,SCPI:99.0 FV:V1.2.7\n'

In [111]:
def barrido_f(f_min,f_max,n,scale='log'):
    if scale == 'log':
        freqs = np.logspace(f_min,f_max,n)
    elif scale == 'lin':
        freqs = np.linspace(f_min,f_max,n)
    else:
        print('Escala inválida')
        return
    time_s = time.time()
    for freq in freqs:
        fungen.write(f'SOURCE1:FREQ {freq}')
        time.sleep(1)
        osci.write('AUTOS exec')
        time.sleep(6)
        osci.write('DAT SOU: CH1')
        xze, xin = osci.query_ascii_values('WFMPRE:XZE?;XIN?', separator=';')
        
        osci.write('ACQ:STATE STOP')
        yze1, ymu1, yoff1 = osci.query_ascii_values('WFMPRE:YZE?;YMU?;YOFF?;', separator=';')
        osci.write('DAT:SOU CH2')
        yze2, ymu2, yoff2 = osci.query_ascii_values('WFMPRE:YZE?;YMU?;YOFF?;', separator=';')
        
        osci.write('DAT:ENC RPB')
        osci.write('DAT:WID 1')
        
        osci.write("DAT:SOU CH1")
        data1 = osci.query_binary_values('CURV?', datatype='B', container=np.array)
        osci.write("DAT:SOU CH2")
        data2 = osci.query_binary_values('CURV?', datatype='B', container=np.array)
        tiempo = xze + np.arange(len(data1)) * xin
        data1v = (data1 - yoff1) * ymu1 + yze1
        ata2v = (data2 - yoff2) * ymu2 + yze2
        
        #data1v = data1 * ymu1 + yze1
        #data2v = data2 * ymu2 + yze2
        osci.write('ACQ:STATE RUN')
        
        df = pd.DataFrame({'t':tiempo,'V_e':data1v,'V_m':data2v})        
        df.to_csv(f'plots seke/{np.round(freq,0)}.csv',index=False)

In [126]:
barrido_f(1000,1000000,10,scale='lin')

In [120]:
df = pd.read_csv('C:\Users\publico.LABORATORIOS\Downloads\plots seke\'med 1 (4,6,10)'\1000000.0.csv´ )

SyntaxError: unexpected character after line continuation character (Temp/ipykernel_14556/1689937074.py, line 1)